# Imports

In [1]:
from scripts.datasets import census_sex_type
from scripts.bmlp import ScBMLPClassifier, Config

n_cells = 20_000
n_genes = 5_000
adata, train_ds, val_ds, test_ds = census_sex_type(n_cells=n_cells, n_genes=n_genes)

cfg = Config(d_input=adata.n_vars, d_hidden=64, d_output=2)
model = ScBMLPClassifier(cfg)
model.fit(train_ds, val_ds)

ImportError: cannot import name 'census_sex_type' from 'scripts.datasets' (/Users/kamal/code/projects/mi/bmlp/bmlp-env/lib/python3.9/site-packages/scripts/datasets.py)

In [2]:
import scripts

In [3]:
help(scripts.datasets)

Help on module scripts.datasets in scripts:

NAME
    scripts.datasets

CLASSES
    torch.utils.data.dataset.Dataset(typing.Generic)
        CellFreqDataset
        CellTypeDataset
    
    class CellFreqDataset(torch.utils.data.dataset.Dataset)
     |  CellFreqDataset(adata, indices: numpy.ndarray, freq_key: str = 'X_freq', device: str = 'cpu')
     |  
     |  PyTorch dataset for single-cell frequency regression tasks.
     |  
     |  Method resolution order:
     |      CellFreqDataset
     |      torch.utils.data.dataset.Dataset
     |      typing.Generic
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]
     |  
     |  __init__(self, adata, indices: numpy.ndarray, freq_key: str = 'X_freq', device: str = 'cpu')
     |      Initialize self.  See help(type(self)) for accurate signature.
     |  
     |  __len__(self) -> int
     |  
     |  --------------------------------------------